In [ ]:
# you'll need to import a module we use every week
import pandas as pd

filepath = "https://github.com/data-to-insight/ERN-sessions/raw/main/data/903_xlsx_errors.xlsx"

# read in the excel, reading in every sheet
dfs = pd.read_excel(filepath, sheet_name=None)

# dict_1 = {"Key_1":"value_1"}
# To look at the dictionary keys use
# dfs.keys()

# start to investigate the data
# access the dict by giving dict name [key name]
dfs['header'].info()

dfs['header'].head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2676 entries, 0 to 2675
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   CHILD   2676 non-null   object 
 1   SEX     2676 non-null   int64  
 2   DOB     2676 non-null   object 
 3   ETHNIC  2676 non-null   object 
 4   UPN     2676 non-null   object 
 5   MOTHER  1 non-null      float64
 6   MC_DOB  1 non-null      object 
dtypes: float64(1), int64(1), object(5)
memory usage: 146.5+ KB


In [3]:
# .notna()
# .isna()
# 102 - date of birth is not a valid date

header = dfs['header']

# header.info()
# header.head() # its' d/m/y

# coercing errors returns NATs for invalid dates
# if I slice out all NaTs - I have all the invalid dates!
header['DOB_dt'] = pd.to_datetime(header['DOB'], format="%d/%m/%Y", errors='coerce')


# invalid_dobs = header['DOB'].notna()
# error_rows = header[~invalid_dobs]

# .isna() returns true where the value in a row in a column is an Na
invalid_dobs = header['DOB'].isna()
error_rows = header[invalid_dobs]

error_rows

,CHILD,SEX,DOB,ETHNIC,UPN,MOTHER,MC_DOB,DOB_dt


In [7]:
# 115 - Date of Local Authority's (LA) decision that a child should be placed for adoption is not a valid date."
# table is placed_for_adoption
p4a = dfs['placed_for_adoption']
# p4a.info()

# covert column is DATE_PLACED to datetime
p4a['DATE_PLACED_dt'] = pd.to_datetime(p4a['DATE_PLACED'], format="%d/%m/%Y", errors="coerce")

# slice the dataframe for invalid dates
invalid_date = p4a['DATE_PLACED_dt'].isna()

error_rows = p4a[invalid_date]
error_rows

,CHILD,DOB,DATE_PLACED,DATE_PLACED_CEASED,REASON_PLACED_CEASED,DATE_PLACED_dt
29,bad_entry_1,Not a date,NaT,This also isn't a date,oop,NaT


In [14]:
# .isin()
# 103 - The ethnicity code is either not valid or has not been entered.

ethnicity_codes = [
        "WBRI",
        "WIRI",
        "WOTH",
        "WIRT",
        "WROM",
        "MWBC",
        "MWBA",
        "MWAS",
        "MOTH",
        "AIND",
        "APKN",
        "ABAN",
        "AOTH",
        "BCRB",
        "BAFR",
        "BOTH",
        "CHNE",
        "OOTH",
        "REFU",
        "NOBT",
    ]

valid_ethnicity = header['ETHNIC'].isin(ethnicity_codes)


error_rows = header[~valid_ethnicity]

error_rows

,CHILD,SEX,DOB,ETHNIC,UPN,MOTHER,MC_DOB,DOB_dt
2675,bad_entry_1,7,oops!,Alien,UPDOG,NaN,NaN,NaT


In [ ]:
# 114 - Data entry to record the status of former carer(s) of an adopted child is invalid.
# When we do an isin we need to check the datatype of the list is the same as the values
# valid codes
code_list = ["0", "1"]

# table is ad1
ad1 = dfs['ad1']

# converts FOSTER_CARE column to a string datatype
ad1['FOSTER_CARE'] = ad1['FOSTER_CARE'].astype('str')

# column is FOSTER_CARE
valid_codes = ad1['FOSTER_CARE'].isin(code_list)

error_rows = ad1[~valid_codes]

error_rows

,CHILD,DOB,DATE_INT,DATE_MATCH,FOSTER_CARE,NB_ADOPTR,SEX_ADOPTR,LS_ADOPTR
29,bad_entry,datedate,notdate,datenot,777,Q,Will,let's go


In [11]:
# 184 - Date of decision that a child should be placed for adoption is before the child was born.

df = pd.merge(p4a, header, how='left', on='CHILD', suffixes=[None, "_header"])

# to specify certain columns to join you could use
#df = pd.merge(p4a, header[['CHILD', 'DOB_dt']], how='left', on='CHILD', suffixes=[None, "_header"])

df['DOB'] = pd.to_datetime(df['DOB'], format="%d/%m/%Y", errors='coerce')
df['DATE_PLACED'] = pd.to_datetime(df['DATE_PLACED'], format="%Y-%m-%d", errors='coerce')

placed_before_birth = (df['DATE_PLACED'] >= df['DOB']) | df['DATE_PLACED'].isna()

error_rows = df[~placed_before_birth]

error_rows



,CHILD,DOB,DATE_PLACED,DATE_PLACED_CEASED,REASON_PLACED_CEASED,DATE_PLACED_dt,SEX,DOB_header,ETHNIC,UPN,MOTHER,MC_DOB,DOB_dt


In [12]:
# 302 - First episode starts before child was born.
# tables are header and episodes
# DOB from header and DECOM from episodes

episodes = dfs['episodes']

episodes['DECOM'] = pd.to_datetime(episodes['DECOM'], format="%Y-%m-%d", errors='coerce')

# sorts values
episodes = episodes.sort_values(['CHILD', 'DECOM'], ascending=True)

# drops duplicates
episodes.drop_duplicates('CHILD', keep='first', inplace=True)

df = pd.merge(episodes, header, how='left', on='CHILD', suffixes=['_epi', '_hdr'])

error_rows = df[df['DECOM'] < df['DOB']]
error_rows

,CHILD,DECOM,RNE,LS,CIN,PLACE,PLACE_PROVIDER,DEC,REC,REASON_PLACE_CHANGE,HOME_POST,PL_POST,URN,SEX,DOB,ETHNIC,UPN,MOTHER,MC_DOB,DOB_dt


In [ ]:
# 188 - Child is aged under 4 years at the end of the year but a Strengths and Difficulties (SDQ) score or a reason for no SDQ score has been completed
oc2 = dfs['oc2']

oc2['DOB'] = pd.to_datetime(oc2['DOB'], format="%d/%m/%Y", errors='coerce')

collection_end = pd.to_datetime('31/03/2020', dayfirst=True)

oc2['4bday'] = oc2['DOB'] + pd.DateOffset(years=4)

error_mask = (oc2['4bday'] > collection_end) & (oc2['SDQ_SCORE'].notna() | oc2['SDQ_REASON'].notna())

error_rows = oc2[error_mask]

error_rows

# 189 - Child is aged 17 years or over at the beginning of the year, but an Strengths
#  and Difficulties (SDQ) score or a reason for no Strengths and Difficulties (SDQ) score has been completed.

# 01/04/2020
# age is 17 


In [ ]:
# 189 - Child is aged 17 years or over at the beginning of the year, but an Strengths
#  and Difficulties (SDQ) score or a reason for no Strengths and Difficulties (SDQ) score has been completed.

collection_start = pd.to_datetime('01/04/2019', dayfirst=True)

over_17_start = oc2['DOB'] + pd.DateOffset(years=17) < collection_start
sdq_score_reason = oc2['SDQ_SCORE'].notna() | oc2['SDQ_REASON'].notna()

error_mask = over_17_start & sdq_score_reason

error_rows = oc2[error_mask]
error_rows